In [4]:
import sqlite3
import csv
from datetime import datetime

In [5]:
# Step 1: Create SQLite Database
conn = sqlite3.connect('LMS.db')
cursor = conn.cursor()

In [6]:
# Create the 'tasks' table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS tasks (
        taskId INTEGER PRIMARY KEY AUTOINCREMENT,
        taskArea TEXT CHECK(taskArea IN ('Question', 'Assessment', 'Content', 'Course', 'Event')),
        taskDeadline DATETIME,
        taskTitle TEXT,
        taskRequirement TEXT,
        isDeleted BOOLEAN,
        createdBy INTEGER,
        creatorEmail TEXT,
        assigneeId INTEGER,
        assigneeEmail TEXT,
        createdOn TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    ''')

In [7]:
conn.commit()
conn.close()

In [8]:

conn = sqlite3.connect('LMS.db')
cursor = conn.cursor()

# Open the CSV file and read data
with open('task.csv', 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        # Insert data into the 'tasks' table
        cursor.execute('''
        INSERT INTO tasks (taskArea, taskDeadline, taskTitle, taskRequirement, isDeleted, 
                                createdBy, creatorEmail, assigneeId, assigneeEmail)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (row['taskArea'], row['taskDeadline'], row['taskTitle'], row['taskRequirement'], 
                row['isDeleted'], row['createdBy'], row['creatorEmail'], row['assigneeId'], row['assigneeEmail']))
    
conn.commit()
conn.close()



In [9]:
conn.commit()
conn.close()

ProgrammingError: Cannot operate on a closed database.

In [12]:
# Create the 'users' table
conn = sqlite3.connect('LMS.db')
cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS users (
        userId INTEGER PRIMARY KEY AUTOINCREMENT,
        userName TEXT,
        userEmail TEXT,
        userPassword TEXT,              
        userRole TEXT CHECK(userRole IN ('Admin', 'Author', 'Instructor', 'Learner'))
    )
    ''')
conn.commit()
conn.close()

In [13]:
conn = sqlite3.connect('LMS.db')
cursor = conn.cursor()

# Open the CSV file and read data
with open('user.csv', 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        # Insert data into the 'tasks' table
        cursor.execute('''
        INSERT INTO users (userName, userEmail, userPassword, userRole)
        VALUES (?, ?, ?, ?)
        ''', (row['userName'], row['userEmail'], row['userPassword'], row['userRole']))
    
conn.commit()
conn.close()

In [32]:
conn.close()

In [16]:
from sqlalchemy import create_engine, Column, Integer, String, Text, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
import sqlalchemy.orm

# Define the base model class
Base = sqlalchemy.orm.declarative_base()

# Define the Course model
class Course(Base):
    __tablename__ = 'course'
    id = Column(Integer, primary_key=True)
    title = Column(String(100), nullable=False)
    description = Column(Text, nullable=False)
    category = Column(String(100), nullable=False)
    sub_category = Column(String(100), nullable=False)
    blocks = relationship('Block', backref='course', lazy=True)

# Define the Block model
class Block(Base):
    __tablename__ = 'block'
    id = Column(Integer, primary_key=True)
    title = Column(String(100), nullable=False)
    course_id = Column(Integer, ForeignKey('course.id'), nullable=False)
    parent_block_id = Column(Integer, ForeignKey('block.id'), nullable=True)
    children = relationship('Block')
    contents = relationship('Content', backref='block', lazy=True)

# Define the Content model
class Content(Base):
    __tablename__ = 'content'
    id = Column(Integer, primary_key=True)
    title = Column(String(100), nullable=False)
    text = Column(Text, nullable=False)
    block_id = Column(Integer, ForeignKey('block.id'), nullable=False)

# Create an engine that stores data in the local directory's LMS.db file.
engine = create_engine('sqlite:///LMS.db')

# Create all tables in the engine. This is equivalent to "Create Table" statements in raw SQL.
Base.metadata.create_all(engine)

/var/folders/t9/8_80rddn50dfxscv_skr6yph0000gn/T/ipykernel_4675/2684364068.py:6: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [15]:
engine = create_engine('sqlite:///LMS.db')

# Drop specific tables. Replace `Course`, `Block`, `Content` with the actual model classes.
Course.__table__.drop(engine)
Block.__table__.drop(engine)
Content.__table__.drop(engine)

print("Selected tables have been deleted.")

Selected tables have been deleted.
